## Google Colab Introdução/dicas

Referências: [Hello Colab ](https://colab.research.google.com/notebooks/welcome.ipynb#scrollTo=-Rh3-Vt9Nev9),[External Date](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=3Jv6jh6HEpP8) , [Google Colab Free GPU Tutorial](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d).

Desenvolvido por [Edresson Casanova](https://github.com/Edresson) para o grupo de Inteligência Artificial [AI-IPB-UTFPR](https://github.com/AI-IPB-UTFPR).

## Conteúdo

>[Google Colab Introdução/dicas](#scrollTo=kumrB-3tUEXI)

>>[Conteúdo](#scrollTo=YzjDL7QhdAcw)

>[Introdução](#scrollTo=SgScHZc1JHQX)

>>[Qual a CPU disponivel?](#scrollTo=AsjCicSKyqOd)

>>[Quanto de Memoria Ram tenho disponivel ?](#scrollTo=TvLFWjzLzJzD)

>[Executar comandos shell](#scrollTo=8PGnueugICKb)

>[Configurando GPU para uso](#scrollTo=S6wB39YKriEQ)

>[Uploading files from your local file system](#scrollTo=BaCkyg5CV5jF)

>[Google Drive Integração](#scrollTo=plupzMLZnQ1Q)



# Introdução

O Google Colab é   um projeto de pesquisa do Google criado para ajudar a disseminar o aprendizado de máquina. É um ambiente de Jupyter notebook  que não requer configuração para usar e é executado inteiramente na nuvem.O Jupyter é, segundo o site do projeto, uma aplicação web de código aberto que permite criar e compartilhar documentos que contenham código vivo, equações, visualizações e texto explicativo. Ele é um dos principais ambientes para a análise de dados, onde você pode usar não apenas Python, mas várias outras linguagens como você pode ver [aqui](https://github.com/jupyter/jupyter/wiki/Jupyter-kernels).

Os cadernos colaborativos são armazenados no Google Drive e podem ser compartilhados exatamente como você faria com o Google Docs or Sheets.

Basicamente no Google Colab o Jupyter roda em uma vps, cada conta possui uma vps propria, que após alguns minutos sem uso é formatada, para assim diminuir o uso de Disco, além de ser totalmente free, o google colab fornece GPU (Tesla K80) para melhorar a performance de treinamento, o uso maximo permitido por dia é de 12 horas.


## Qual a CPU disponivel?
Você pode visualizar executando o codigo abaixo

In [0]:
!cat /proc/cpuinfo

## Quanto de Memoria Ram tenho disponivel ?
Você pode obter essa informação executando o codigo abaixo.

In [0]:
!cat /proc/meminfo

# Executar comandos shell

Para executar comandos shell no  Colab utiliza-se a expressão **!** antes do comando shell, exemplos:

In [0]:

!ls
!uname -a 
!cat /etc/lsb-release # ver a distribuição do linux 
!apt update #atualiza o repositorio
!pip install --upgrade pip #upgrade pip

# Configurando GPU para uso
Por padrão o Colab utiliza CPU, para alterar essa configuração basta ir no menu **Runtime>Change runtime type** , altere a opção **Hardware Accerelator** para GPU, nesta aba ainda é possivel alterar a versão do python, que deseja utilizar para rodar seus codigo no Jupyter. Para testar se a GPU está sendo usada execute o codigo abaixo , se tudo ocorrer bem deve mostrar 2 Devices, CPU e GPU, incluindo no final o nome da GPU.

Nota: Se quando você mudar as configurações de CPU para GPU você receber o seguinte erro 

```
Failed to assign a backend
No backend with GPU available. Would you like to use a runtime with no accelerator?

```
significa que todas as GPU disponibilizadas pelo google estão sendo usadas, você deve tentar mais tarde.




In [0]:

from tensorflow.python.client import device_lib
device_lib.list_local_devices()


#Upload Local de arquivos

`files.upload` retorna um dicionário dos arquivos que foram carregados.
O dicionário é codificado pelo nome do arquivo, o valor é o dado que foi carregado. Execute o codigo abaixo para testar o upload local.

Nota: essa função não funciona no firefox, tente usando google chrome  ou chromium.

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Após o upload é necessário utilizar a variavel uploaded para abrir o arquivo, desta forma é necessário fazer upload novamente a cada nova execução do script. Para driblar isso existe uma função não oficial, que basicamente pega a lista da variavel uploaded e cria um novo arquivo porém esse no diretorio atual, dessa forma é possivel abrir normalmente sem o uso da variavel uploaded,  outra vantagem que só será necessário upar novamente o arquivo se você sair e sua maquina foi formatada, ou provavelmente se o seu periodo de 12 horas for esgotado.

In [0]:
# upload files, usando está função o arquivo é baixado normalmente dispensando o uso da variavel uploaded para abrir o arquivo. .
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

print("arquivos upados:",upload_files())

# Google Drive Integração

A ideia é rodar a aplicação sem se preocupar com salvar seus arquivos, nem mesmo baixar, a ideia é montar o google Drive no Colab, Primeiro crie uma pasta no seu google Drive, eu criei a pasta **DeepLearning**. Após é necessário instalar algumas dependências no Colab, rode o codigo abaixo, após irá aparecer um campo para você digitar, acima deste campo tem um link, clique nele e de as permissões solicitadas, após dar as premissões abrirá uma pagina contendo um codigo copie o codigo, cole no campo abaixo do link, e precione Enter.

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Se você receber o seguinte Warning **Warning: apt-key output should not be parsed (stdout is not a terminal)**, significa que você já autenticou, dessa forma apenas ignore este Warning e passe para o proximo passo.

Agora já estamos com as permissões e dependências, só precisamos criar uma pasta e montar nosso drive :), execute o codigo abaixo.


In [0]:
! mkdir -p drive 
! google-drive-ocamlfuse drive


Você pode acessar as pastas do seu drive, basicamente todo seu drive está montado na pasta drive/ , é importante ressaltar que o comando **!cd** não funciona corretamente no Colab, mas podemos navegar nos diretorios usando a biblioteca **os** do python, execute o codigo abaixo e você entrará na pasta de seu drive.

In [0]:
# necessário para navegar nos diretorios do Colab
import os

!mkdir drive/DeepLearning


os.chdir('drive/DeepLearning') # Entra na pasta DeepLearning criada no Google Drive. 
!echo "arquivo teste, criado para testar a Integração do Colab com o Google Drive" > test.txt # cria o arquivo test.txt
print('O arquivo test.txt foi criado na pasta DeepLearning de seu Drive, verifique!')
!ls # lista os arquivos da pasta DeepLearning
os.chdir('..')#retorna para a pasta drive
os.chdir('..')#retorna para a pasta raiz do Colab


Agora vamos excluir o arquivo **test.txt** criado em nosso Drive, após listando os arquivos do diretorio DeepLearning do nosso Drive, **Cuidado com  **rm -rf **você pode apagar diretorio inteiros, incluindo subpastas de seu drive..** 

In [0]:
!rm drive/DeepLearning/test.txt
!ls -l drive/DeepLearning/ #listando os arquivos da pasta DeepLearning, sem estar no diretorio

Suponho que você tenha um projeto que está rodando em seu computador pessoal, para passar para o Colab você apenas deve upar na pasta DeepLearning de seu Drive, após supondo que o script se chama **exemplo.py**, você pode executar sem mudar de diretorio como no exemplo:

In [0]:
!python3 drive/DeepLearning/exemplo.py

È importante lembrar que a autenticação com o Drive deve ser feita toda vez que você entrar em sua conta após um periodo inativo, isso ocorre pois a maquina é formata se estiver alguns minutos sem trabalhar. Para saber se é necesário autenticar e montar novamente seu Drive você pode dar um **!ls** se a pasta **drive** não existir, será necessario.